Extracting youtube video and its audio  
using pytube package of python.

In [ ]:
! pip install pytube
! pip install moviepy

In [41]:
from pytube import YouTube
from moviepy.editor import VideoFileClip


getting stream value of video from youtube using pytube

In [13]:
# url = 'https://www.youtube.com/watch?v=Sby1uJ_NFIY'
url = 'https://www.youtube.com/watch?v=CeSAjK2CBEA&t=81s'

yt_vid = YouTube(url) 

# progressive streams have video and audio in a single file 
# but are only for upto 720p
yt_vid.streams.filter(progressive=True)


[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">]

download video from youtube url

In [42]:
def get_video(yt_url):
    try:
        video = YouTube(yt_url)
        video.streams.get_by_itag(22).download(filename='video.mp4')
        print('Video succesfully downloaded from Youtube')
    except Exception as e:
        print(f'Failed to download Youtube video \nerror : {e}')


Extracting audio from downloaded video

In [44]:
def audio_from_video(video_path):
    try:
        video = VideoFileClip(video_path)
        audio = video.audio
        audio.write_audiofile('audio.mp3')   
        video.close()
        audio.close()
    except Exception as e:
        print(f'Failed to extract audio from video \nerror : {e}')


In [47]:
url = 'https://www.youtube.com/watch?v=CeSAjK2CBEA&t=81s'
video_path = './video.mp4'

get_video(url)
audio_from_video(video_path)


Video succesfully downloaded from Youtube
MoviePy - Writing audio in audio.mp3


MoviePy - Done.
